In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import lxml
import pandas as pd
from urllib.robotparser import RobotFileParser
from concurrent.futures import ThreadPoolExecutor

In [40]:
dataLinks = [] # Script pro extrakce odkazů na jetnotlivé inzeraty
headers = {
    'User-Agent': 'MyScraperBot',
}

# Načtení robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

def process_page(p):
    try:
        url = f"https://www.tipcars.com/nove/?hlavni-mesto-praha&10000-kc&palivo-multiple=benzin%2Ccng%2Clpg%2Cnafta&str={p}"
        
        sleep(1) # přidání zpoždění
        
        print(p) # zobrazí, které stránky byly zpracovány
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], url):
            print(f"Přístup k {url} není povolen podle robots.txt")
            return

        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')

        # Získání odkazů na jednotlivé inzeráty
        links = soup.find('div', class_='sloupec_s_inzeratama').findAll('a', href=True)
        for link in links:
            odkaz = f"https://www.tipcars.com{link.get('href')}"
            dataLinks.append([odkaz])

    except Exception as e:
        print(f"Chyba při zpracování stránky {p}: {e}")

# Vytvoření ThreadPoolExecutor pro vícevláknové zpracování
with ThreadPoolExecutor() as executor:
    
    # Spuštění funkce process_page s argumenty od 1 do n ve vícevláknovém režimu
    executor.map(process_page, range(1, 181))


21
3
12
11
8
4
5
9
6
7

10
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
10099

101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [36]:
print(len(dataLinks))

17338


In [41]:
data = [] # Script pro extrakce dat z aut s palivem benzina 
failed_links = []

headers = {
    'User-Agent': 'MyScraperBot'
}

# Načtení souboru robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

# Omezení počtu odkazů k zpracování
start = 0
end = 1000
dataLinksLimited = dataLinks[start:end]
limit = end - start

def process_link(dataLink):
    global data
    global failed_links
    try:
        uri = dataLink[0]
        sleep(3)
        print(uri)
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], uri):
            print(f"Přístup k {uri} není povolen podle robots.txt")
            return

        f = requests.get(uri, headers=headers)
        pol = BeautifulSoup(f.text, 'lxml')

        # Extrahování informací o vozidle (značka, rok vyroby, stav, atd.)
        odkaz = uri  
        auto = pol.find('h1', class_='fs-nejvetsi float-l mb-20').find('b').text
        vyrobeno = pol.findAll('div', class_='radek')[1].findAll('div', class_='sl-6')[1].text
        zaruka = pol.findAll('div', class_='radek')[2].findAll('div', class_='sl-6')[1].text
        stav = ' '.join(pol.findAll('div', class_='radek')[3].findAll('div', class_='sl-6')[1].text.strip().split()).replace('prověřit technikem', '').strip()
        tachometr = ' '.join(pol.findAll('div', class_='radek')[4].findAll('div', class_='sl-6')[1].text.strip().split()).replace('zobrazit historii vozidla', '').strip()
        stk = pol.findAll('div', class_='radek')[5].findAll('div', class_='sl-6')[1].text
        objem = pol.findAll('div', class_='radek')[6].findAll('div', class_='sl-6')[1].text
        vykon = pol.findAll('div', class_='radek')[7].findAll('div', class_='sl-6')[1].text
        moment = pol.findAll('div', class_='radek')[8].findAll('div', class_='sl-6')[1].text
        palivo = pol.findAll('div', class_='radek')[9].findAll('div', class_='sl-6')[1].text
        karoserie = pol.findAll('div', class_='radek')[10].findAll('div', class_='sl-6')[1].text
        pocet_dveri = pol.findAll('div', class_='radek')[11].findAll('div', class_='sl-6')[1].text
        pocet_mist = pol.findAll('div', class_='radek')[12].findAll('div', class_='sl-6')[1].text
        barva = pol.findAll('div', class_='radek')[13].findAll('div', class_='sl-6')[1].text.strip()
        typ = 'Nové' # Zaleži na tom jaký filtr jste nastavil
        cena = pol.find('div', class_='fs-velke').find('b').text
        
        # Vložení dat do data[]
        data.append([odkaz, auto, vyrobeno, zaruka, stav, tachometr, stk, objem, vykon, moment, palivo, karoserie, pocet_dveri, pocet_mist, barva, typ, cena])

        # Zastavení zpracování po dosažení stanoveného limitu
        if len(data) >= limit:
            return

    except Exception as e:
        print(f"Chyba při zpracování stránky {dataLink}: {e}")
        failed_links.append(dataLink)
        
# Procházení odkazů na jednotlivé nabídky paralelně
with ThreadPoolExecutor() as executor:
    executor.map(process_link, dataLinksLimited)

# Opětovné zpracování odkazů, u kterých došlo k chybě při prvním pokusu
retries = 2 # počet pokusu

for _ in range(retries):
    if not failed_links:
        break

    sleep(3)  # zpoždění

    failed_links_copy = failed_links.copy()
    failed_links = []

    with ThreadPoolExecutor() as executor:
        executor.map(process_link, failed_links_copy)


header = ['odkaz', 'auto', 'vyrobeno', 'záruka', 'stav', 'tachometr', 'STK', 'objem', 'výkon', 'moment', 'palivo', 'karoserie', 'počet_dveří', 'počet_míst', 'barva', 'typ','cena']

# Vytvoření DataFrame s použitím načtených dat a zadaných sloupců
df = pd.DataFrame(data, columns=header)

# Uložení DataFrame do souboru CSV
df.to_csv('/Users/adaye/Desktop/tipcars_benzin_nové.csv', sep=';', encoding='utf8')

https://www.tipcars.com/peugeot-308/hatchback/benzin/peugeot-308-active-pack-1-2-130-man6-38268547.html
https://www.tipcars.com/seat-arona/suv/benzin/seat-arona-xperience-1-0-tsi-110k-33878237.html
https://www.tipcars.com/seat-arona/suv/benzin/seat-arona-xperience-1-0-tsi-110k-33878242.html
https://www.tipcars.com/opel-crossland/suv/benzin/opel-crossland-elegance-1-2-turbo-96kw-130k-12985516.html
https://www.tipcars.com/skoda-scala/hatchback/benzin/skoda-scala-style-1-0-tsi-81-kw-6-stup-m-30031503.html
https://www.tipcars.com/seat-arona/suv/benzin/seat-arona-xperience-1-0-tsi-110k-33878386.html
https://www.tipcars.com/skoda-kamiq/suv/benzin/skoda-kamiq-style-1-0-tsi-81-kw-6-stup-m-30031428.html
https://www.tipcars.com/skoda-scala/hatchback/benzin/skoda-scala-style-1-0-tsi-81-kw-6-stup-m-30031413.html
https://www.tipcars.com/skoda-kamiq/suv/benzin/skoda-kamiq-style-1-0-tsi-81-kw-6-stup-m-30031499.html
https://www.tipcars.com/opel-crossland/suv/benzin/opel-crossland-elegance-1-2-turbo-96

https://www.tipcars.com/kia-stonic/hatchback/benzin/kia-stonic-1-0-top-5840238.html
https://www.tipcars.com/dacia-duster/suv/nafta/dacia-duster-extreme-blue-dci-115-4x4-33878458.html
https://www.tipcars.com/peugeot-308/kombi/benzin/peugeot-308-sw-active-pack-1-2-130-man6-38268548.html
https://www.tipcars.com/suzuki-vitara/suv/benzin/suzuki-vitara-1-4-premium-vuz-na-objednavku-53160061.html
https://www.tipcars.com/peugeot-208/hatchback/benzin/peugeot-208-gt-1-2-pt-100k-eat8-8382973.html
https://www.tipcars.com/kia-ceed/hatchback/benzin/kia-ceed-1-5-t-gdi-gpf-7dct-12871980.html
https://www.tipcars.com/ssangyong-korando/sedan/benzin/ssangyong-korando-1-5-tgdi-style-mt-skladem-54280018.html
https://www.tipcars.com/ssangyong-korando/suv/benzin/ssangyong-korando-1-5-tgdi-style-mt-skladem-54280019.html
https://www.tipcars.com/opel-mokka/suv/benzin/opel-mokka-elegance-1-2t-74kw-100hp-mt6-5151715.html
https://www.tipcars.com/opel-combo/mpv/benzin/opel-combo-life-edition-1-2t-81kw-110hp-5150420.

https://www.tipcars.com/peugeot-308/hatchback/benzin/peugeot-308-allure-pack-1-2-puretech-130-s-11114912.html
https://www.tipcars.com/peugeot-308/hatchback/benzin/peugeot-308-active-pack-1-2-puretech-130-s-11116229.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-active-pack-puretech130-eat8-53537338.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-active-pack-puretech130-eat8-53537385.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-active-pack-puretech130-man6-53537339.html
https://www.tipcars.com/peugeot-308/kombi/benzin/peugeot-308-sw-active-pack-puretech130-53537332.html
https://www.tipcars.com/peugeot-308/kombi/benzin/peugeot-308-sw-active-pack-puretech130-53537374.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-allure-puretech130-man6-53537312.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-active-pack-puretech130-man6-53537340.html
https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-

https://www.tipcars.com/toyota-aygo/hatchback/benzin/toyota-aygo-1-0-x-style-tech-vision-32210286.html
https://www.tipcars.com/fiat-500/hatchback/benzin/fiat-500-1-0-firefly-70k-cult-sklade-23140159.html
https://www.tipcars.com/toyota-aygo/suv/benzin/toyota-aygo-style-50507130.html
https://www.tipcars.com/toyota-aygo/suv/benzin/toyota-aygo-x-style-50507129.html
https://www.tipcars.com/skoda-octavia/kombi/nafta/skoda-octavia-2-0-tdi-110-kw-style-9180410.html
https://www.tipcars.com/renault-clio/hatchback/benzin/renault-clio-equllibre-tce-90-33878453.html
https://www.tipcars.com/dacia-sandero/suv/lpg/dacia-sandero-100ps-lpg-kamera-vyhr-sedacky-33878481.html
https://www.tipcars.com/kia-picanto/hatchback/benzin/kia-picanto-1-2-dpi-5amt-gt-line-2023-12873284.html
https://www.tipcars.com/dacia-jogger/kombi/lpg/dacia-jogger-essential-100tce-eco-g-5-mist-33878478.html
https://www.tipcars.com/renault-clio/hatchback/benzin/renault-clio-equilibre-tce-90-30511179.html
https://www.tipcars.com/opel-

https://www.tipcars.com/peugeot-2008/suv/benzin/peugeot-2008-allure-puretech130-eat8-53537316.html
https://www.tipcars.com/volkswagen-t-roc/suv/benzin/volkswagen-t-roc-life-1-5-tsi-110-kw-9185729.html
https://www.tipcars.com/volkswagen-golf/hatchback/benzin/volkswagen-golf-1-5-tsi-style-cr-dph-51600425.html
https://www.tipcars.com/suzuki-vitara/suv/benzin/suzuki-vitara-1-4-elegance-4x4-vuz-skladem-53160062.html
https://www.tipcars.com/kia-xceed/suv/benzin/kia-xceed-1-5-t-gdi-gpf-7dct-top-12872994.html
https://www.tipcars.com/toyota-proace/pick-up/nafta/toyota-proace-shuttle-long-1-5-d-4d-100k-6-12873001.html
https://www.tipcars.com/toyota-corolla/sedan/benzin/toyota-corolla-1-5-mt-comfort-style-tech-12872017.html
https://www.tipcars.com/citroen-berlingo/kombi/nafta/citroen-berlingo-1-5-bluehdi-130-s-s-man6-feel-47345153.html
https://www.tipcars.com/kia-sportage/suv/benzin/kia-sportage-1-6-comfort-5840233.html
https://www.tipcars.com/peugeot-308/hatchback/nafta/peugeot-308-active-pack-1

...

https://www.tipcars.com/kia-proceed/kombi/benzin/kia-proceed-1-6-gt-5840245.html
https://www.tipcars.com/kia-proceed/kombi/benzin/kia-proceed-1-6-gt-5840220.html
https://www.tipcars.com/peugeot-408/sedan/benzin/peugeot-408-allure-pack-1-2pt-130-eat8-8383664.html
https://www.tipcars.com/ford-kuga/hatchback/benzin/ford-kuga-st-line-x-5dverova-1-5-ecobo-52750045.html
https://www.tipcars.com/ds-automobiles-ds4/hatchback/nafta/ds-automobiles-ds4-1-5-bluehdi-trocadero-130k-50916176.html
https://www.tipcars.com/peugeot-3008/suv/benzin/peugeot-3008-active-pack-1-2-pt-eat8-8383260.html
https://www.tipcars.com/skoda-karoq/suv/benzin/skoda-karoq-sportline-1-5-tsi-110-kw-7-st-30031294.html
https://www.tipcars.com/peugeot-5008/suv/benzin/peugeot-5008-allure-pack-1-2-pt-130k-man6-38268261.html
https://www.tipcars.com/skoda-karoq/suv/benzin/skoda-karoq-sportline-1-5-tsi-110-kw-7-30031372.html
https://www.tipcars.com/kia-proceed/hatchback/benzin/kia-proceed-skladem-1-6-t-gdi-gpf-7dct-g-12872665.html
h

https://www.tipcars.com/bmw-rada-3/sedan/benzin/bmw-320i-sedan-49132729.html
https://www.tipcars.com/volkswagen-touran/mpv/benzin/volkswagen-touran-hl-1-5-tsi-evo-7dsg-9182678.html
https://www.tipcars.com/bmw-rada-1/hatchback/benzin/bmw-118i-hatchback-mpaket-49133038.html
https://www.tipcars.com/opel-grandland/suv/benzin/opel-grandland-ultimate-1-2t-96kw-130hp-at8-5155534.html
https://www.tipcars.com/kia-sportage/suv/nafta/kia-sportage-1-6-exclusive-5840231.html
https://www.tipcars.com/peugeot-308/kombi/nafta/peugeot-308-sw-gt-1-5-bhdi-130k-eat8-8383394.html
https://www.tipcars.com/peugeot-5008/suv/benzin/peugeot-5008-active-pack-1-2-pt-eat8-8383079.html
https://www.tipcars.com/peugeot-5008/suv/benzin/peugeot-5008-active-pack-puretech130-eat8-53537381.html
https://www.tipcars.com/renault-trafic/minibus/nafta/renault-trafic-l2-equilibre-150k-dci-33878480.html
https://www.tipcars.com/bmw-rada-1/hatchback/benzin/bmw-118i-hatchback-mpaket-49132863.html
https://www.tipcars.com/kia-sportage/

https://www.tipcars.com/volkswagen-passat/kombi/nafta/volkswagen-passat-variant-r-line-2-0-tdi-7dsg-9190304.html
https://www.tipcars.com/volkswagen-passat/kombi/nafta/volkswagen-passat-2-0tdi-147kw-4-motion-elegance-9192658.html
https://www.tipcars.com/isuzu-d-max/pick-up/nafta/isuzu-d-max-double-cab-1-9l-ls-4x4-man-49548212.html
https://www.tipcars.com/peugeot-5008/suv/nafta/peugeot-5008-allure-1-5-bluehdi-130-s-amp-s-11112218.html
https://www.tipcars.com/peugeot-5008/suv/benzin/peugeot-5008-allure-pack-puretech130-eat8-53537384.html
https://www.tipcars.com/peugeot-408/liftback/benzin/peugeot-408-gt-1-2-puretech130-eat8-53537353.html
https://www.tipcars.com/volvo-xc40/suv/benzin/volvo-xc40-b3-fwd-benzin-core-my24-45782309.html
https://www.tipcars.com/mini-cooper/kabriolet/benzin/mini-cooper-cabrio-26140582.html
https://www.tipcars.com/bmw-x1/suv/nafta/bmw-x1-xdrive18d-mpaket-49132974.html
https://www.tipcars.com/volkswagen-tiguan/suv/benzin/volkswagen-tiguan-allspace-life-1-5-tsi-110-

https://www.tipcars.com/peugeot-2008/suv/nafta/peugeot-2008-allure-pack-1-5-bhdi-130k-eat8-8383695.html
https://www.tipcars.com/ford-kuga/hatchback/benzin/ford-kuga-st-line-x-5dverova-1-5-ecobo-52750091.html
https://www.tipcars.com/opel-astra/hatchback/benzin/opel-astra-elegance-hb-1-2-turbo-96kw-13-12984502.html
https://www.tipcars.com/renault-megane/kombi/benzin/renault-megane-megane-grandtour-r-s-line-tce-33878411.html
https://www.tipcars.com/ford-kuga/hatchback/benzin/ford-kuga-st-line-x-5dverova-1-5-ecobo-52750063.html
https://www.tipcars.com/ford-kuga/hatchback/benzin/ford-kuga-st-line-x-5dverova-1-5-ecobo-52750026.html
https://www.tipcars.com/toyota-proace-city/mpv/benzin/toyota-proace-city-1-2-6-m-t-family-long-32210235.html
https://www.tipcars.com/peugeot-308/hatchback/nafta/peugeot-308-gt-1-5-bhdi-130k-eat8-8383755.html
https://www.tipcars.com/volkswagen-t-roc/suv/benzin/volkswagen-t-roc-style-1-5-tsi-110-kw-dsg-9192017.html
https://www.tipcars.com/peugeot-308/kombi/benzin/pe

https://www.tipcars.com/skoda-octavia/liftback/benzin/skoda-octavia-1-5-tsi-110-kw-style-dsg-9186358.html
https://www.tipcars.com/subaru-xv/suv/benzin/subaru-xv-1-6-executive-lineartronic-skl-53140036.html
https://www.tipcars.com/kia-ceed/kombi/benzin/kia-ceed-1-5-gt-line-5840208.html
https://www.tipcars.com/volkswagen-t-roc/hatchback/benzin/volkswagen-t-roc-life-1-5-tsi-110-kw-7dsg-9195801.html
https://www.tipcars.com/kia-sportage/suv/benzin/kia-sportage-1-6-top-5840249.html
https://www.tipcars.com/hyundai-tucson/suv/benzin/hyundai-tucson-my23-1-6-t-gdi-4x2-dct-110-kw-53195938.html
https://www.tipcars.com/bmw-rada-2/benzin/bmw-218i-gc-mpaket-49132422.html
https://www.tipcars.com/bmw-rada-3/sedan/benzin/bmw-318i-limousine-49132752.html
https://www.tipcars.com/bmw-rada-5/kombi/benzin/bmw-540i-xdrive-touring-luxury-lin-49132999.html
https://www.tipcars.com/bmw-rada-3/sedan/benzin/bmw-318i-limousine-49132749.html
https://www.tipcars.com/mini-clubman/kombi/benzin/mini-clubman-cooper-491327

In [51]:
dataLinks = [] # Script pro extrakce odkazů na jetnotlivé inzeraty (hybridní auta)

headers = {
    'User-Agent': 'MyScraperBot',
}

# Načtení robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

def process_page(p):
    try:
        url = f"https://www.tipcars.com/nove/?hlavni-mesto-praha&10000-kc&palivo-multiple=hybridni,hybridni-nafta&str={p}"
        
        sleep(1) # přidání zpoždění
        
        print(p) # zobrazí, které stránky byly zpracovány
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], url):
            print(f"Přístup k {url} není povolen podle robots.txt")
            return

        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')

        # Získání odkazů na jednotlivé inzeráty
        links = soup.find('div', class_='sloupec_s_inzeratama').findAll('a', href=True)
        for link in links:
            odkaz = f"https://www.tipcars.com{link.get('href')}"
            dataLinks.append([odkaz])

    except Exception as e:
        print(f"Chyba při zpracování stránky {p}: {e}")

# Vytvoření ThreadPoolExecutor pro vícevláknové zpracování
with ThreadPoolExecutor() as executor:
    
    # Spuštění funkce process_page s argumenty od 1 do n ve vícevláknovém režimu
    executor.map(process_page, range(1, 5))


3
2
4
1


In [52]:
data = [] # Script pro extrakce dat z aut s palivem hybridní 
failed_links = []

headers = {
    'User-Agent': 'MyScraperBot'
}

# Načtení souboru robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

# Omezení počtu odkazů k zpracování
start = 0
end = 1000
dataLinksLimited = dataLinks[start:end]
limit = end - start

def process_link(dataLink):
    global data
    global failed_links
    try:
        uri = dataLink[0]
        sleep(3)
        print(uri)
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], uri):
            print(f"Přístup k {uri} není povolen podle robots.txt")
            return

        f = requests.get(uri, headers=headers)
        pol = BeautifulSoup(f.text, 'lxml')

        # Extrahování informací o vozidle (značka, rok vyroby, stav, atd.)
        odkaz = uri  
        auto = pol.find('h1', class_='fs-nejvetsi float-l mb-20').find('b').text
        vyrobeno = pol.findAll('div', class_='radek')[1].findAll('div', class_='sl-6')[1].text
        zaruka = pol.findAll('div', class_='radek')[2].findAll('div', class_='sl-6')[1].text
        stav = ' '.join(pol.findAll('div', class_='radek')[3].findAll('div', class_='sl-6')[1].text.strip().split()).replace('prověřit technikem', '').strip()
        tachometr = ' '.join(pol.findAll('div', class_='radek')[4].findAll('div', class_='sl-6')[1].text.strip().split()).replace('zobrazit historii vozidla', '').strip()
        stk = pol.findAll('div', class_='radek')[5].findAll('div', class_='sl-6')[1].text
        objem = pol.findAll('div', class_='radek')[8].findAll('div', class_='sl-6')[1].text
        vykon = pol.findAll('div', class_='radek')[9].findAll('div', class_='sl-6')[1].text
        moment = pol.findAll('div', class_='radek')[10].findAll('div', class_='sl-6')[1].text
        palivo = pol.findAll('div', class_='radek')[11].findAll('div', class_='sl-6')[1].text
        karoserie = pol.findAll('div', class_='radek')[13].findAll('div', class_='sl-6')[1].text
        pocet_dveri = pol.findAll('div', class_='radek')[14].findAll('div', class_='sl-6')[1].text
        pocet_mist = pol.findAll('div', class_='radek')[15].findAll('div', class_='sl-6')[1].text
        barva = pol.findAll('div', class_='radek')[16].findAll('div', class_='sl-6')[1].text.strip()
        typ = 'Nové'
        cena = pol.find('div', class_='fs-velke').find('b').text
        
        # Vložení dat do data[]
        data.append([odkaz, auto, vyrobeno, zaruka, stav, tachometr, stk, objem, vykon, moment, palivo, karoserie, pocet_dveri, pocet_mist, barva, typ, cena])

        # Zastavení zpracování po dosažení stanoveného limitu
        if len(data) >= limit:
            return

    except Exception as e:
        print(f"Chyba při zpracování stránky {dataLink}: {e}")
        failed_links.append(dataLink)
        
# Procházení odkazů na jednotlivé nabídky paralelně
with ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(process_link, dataLinksLimited)

# Opětovné zpracování odkazů, u kterých došlo k chybě při prvním pokusu
retries = 2 # počet pokusu

for _ in range(retries):
    if not failed_links:
        break

    sleep(3)  # zpoždění

    failed_links_copy = failed_links.copy()
    failed_links = []

    with ThreadPoolExecutor() as executor:
        executor.map(process_link, failed_links_copy)


header = ['odkaz', 'auto', 'vyrobeno', 'záruka', 'stav', 'tachometr', 'STK', 'objem', 'výkon', 'moment', 'palivo', 'karoserie', 'počet_dveří', 'počet_míst', 'barva', 'typ', 'cena']

# Vytvoření DataFrame s použitím načtených dat a zadaných sloupců
df = pd.DataFrame(data, columns=header)

# Uložení DataFrame do souboru CSV
df.to_csv('/Users/adaye/Desktop/tipcars_hybridni_nove.csv', sep=';', encoding='utf8')

https://www.tipcars.com/jaguar-e-pace/suv/hybridni/jaguar-e-pace-1-5-p300e-r-dynamic-se-my23-49172690.html
https://www.tipcars.com/land-rover-range-rover-evoque/suv/hybridni/land-rover-range-rover-evoque-1-5-p300e-r-dynamic-hse-my23-49172925.html
https://www.tipcars.com/land-rover-range-rover-evoque/suv/hybridni/land-rover-range-rover-evoque-p300e-r-dynamic-se-awd-aut-49548903.html
https://www.tipcars.com/land-rover-range-rover-evoque/suv/hybridni/land-rover-range-rover-evoque-1-5-p300e-r-dynamic-se-my23-5-49172792.html
https://www.tipcars.com/land-rover-discovery-sport/suv/hybridni/land-rover-discovery-sport-1-5-p300e-r-dyn-se-phev-my23-5-49172871.html
https://www.tipcars.com/volvo-xc60/suv/hybridni/volvo-xc60-t8-awd-recharge-2-0l-310-145-h-12983068.html
https://www.tipcars.com/jeep-wrangler/suv/hybridni/jeep-wrangler-2-0t-4xe-phev-380k-8at-rubicon-12986761.html
https://www.tipcars.com/maserati-ghibli/sedan/hybridni/maserati-ghibli-hybrid-243-kw-volna-spec-25287063.html
https://www.ti

https://www.tipcars.com/suzuki-vitara/suv/hybridni/suzuki-vitara-1-4-elegance-4x4-hybrid-34897181.html
https://www.tipcars.com/renault-arkana/suv/hybridni/renault-arkana-zen-e-tech-145-27182124.html
https://www.tipcars.com/toyota-yaris-cross/suv/hybridni/toyota-yaris-cross-1-5-fhev-executive-50507124.html
https://www.tipcars.com/toyota-yaris-cross/suv/hybridni/toyota-yaris-cross-1-5-fhev-executive-50507152.html
https://www.tipcars.com/suzuki-s-cross/suv/hybridni/suzuki-s-cross-1-4-elegance-hybrid-akce-34897183.html
https://www.tipcars.com/suzuki-swace/kombi/hybridni/suzuki-swace-premium-1-8-cvt-4x2-12873273.html
https://www.tipcars.com/toyota-yaris-cross/cuv/hybridni/toyota-yaris-cross-1-5-hybrid-executive-smart-32210196.html
https://www.tipcars.com/renault-arkana/suv/hybridni/renault-arkana-intens-e-tech-145-27181792.html
https://www.tipcars.com/toyota-yaris-cross/suv/hybridni/toyota-yaris-cross-1-5-fhev-executive-jbl-4x4-50507151.html
https://www.tipcars.com/toyota-yaris-cross/cuv/hy

https://www.tipcars.com/toyota-rav4/suv/hybridni/toyota-rav4-2-5-hybrid-fwd-selection-32210278.html
https://www.tipcars.com/toyota-rav4/suv/hybridni/toyota-rav4-2-5-hybrid-fwd-selection-32210276.html
https://www.tipcars.com/opel-astra/hatchback/hybridni/opel-astra-gs-line-phev-132kw-180hp-at8-5152733.html
https://www.tipcars.com/ford-kuga/hatchback/hybridni/ford-kuga-st-line-x-5dverova-2-5-durat-52750051.html
https://www.tipcars.com/ford-s-max/hatchback/hybridni/ford-s-max-titanium-5m-5dverova-2-5-dur-52750057.html
https://www.tipcars.com/ford-s-max/hatchback/hybridni/ford-s-max-titanium-5m-5dverova-2-5-dur-52750077.html
https://www.tipcars.com/ford-s-max/hatchback/hybridni/ford-s-max-titanium-5m-5dverova-2-5-dur-52750010.html
https://www.tipcars.com/ford-s-max/hatchback/hybridni/ford-s-max-titanium-5m-5dverova-2-5-dur-52750074.html
https://www.tipcars.com/ford-kuga/hatchback/hybridni/ford-kuga-st-line-x-5dverova-2-5-durat-52750033.html
https://www.tipcars.com/ford-s-max/hatchback/hybr

https://www.tipcars.com/bmw-rada-3/sedan/hybridni/bmw-330e-xdrive-sedan-mpaket-49132970.html
https://www.tipcars.com/jaguar-e-pace/suv/hybridni/jaguar-e-pace-p300e-r-dynamic-se-awd-aut-49547544.html
https://www.tipcars.com/bmw-rada-5/kombi/hybridni/bmw-530e-xdrive-touring-mpaket-49132775.html


In [58]:
dataLinks = [] # Script pro extrakce odkazů na jetnotlivé inzeraty (elektro auta)

headers = {
    'User-Agent': 'MyScraperBot',
}

# Načtení robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

def process_page(p):
    try:
        url = f"https://www.tipcars.com/nove/elektro/?hlavni-mesto-praha&10000-kc&str={p}"
        
        sleep(1) # přidání zpoždění
        
        print(p) # zobrazí, které stránky byly zpracovány
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], url):
            print(f"Přístup k {url} není povolen podle robots.txt")
            return

        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')

        # Získání odkazů na jednotlivé inzeráty
        links = soup.find('div', class_='sloupec_s_inzeratama').findAll('a', href=True)
        for link in links:
            odkaz = f"https://www.tipcars.com{link.get('href')}"
            dataLinks.append([odkaz])

    except Exception as e:
        print(f"Chyba při zpracování stránky {p}: {e}")

# Vytvoření ThreadPoolExecutor pro vícevláknové zpracování
with ThreadPoolExecutor() as executor:
    
    # Spuštění funkce process_page s argumenty od 1 do n ve vícevláknovém režimu
    executor.map(process_page, range(1, 6))


1
3
4
5
2


In [59]:
data = [] # Script pro extrakce dat z elektrocarů 
failed_links = []

headers = {
    'User-Agent': 'MyScraperBot'
}

# Načtení souboru robots.txt
robots_url = 'https://www.tipcars.com/robots.txt'
robot_parser = RobotFileParser()
robot_parser.set_url(robots_url)
robot_parser.read()

# Omezení počtu odkazů k zpracování
start = 0
end = 1000
dataLinksLimited = dataLinks[start:end]
limit = end - start

def process_link(dataLink):
    global data
    global failed_links
    try:
        uri = dataLink[0]
        sleep(3)
        print(uri)
        
        # Kontrola pravidel robots.txt
        if not robot_parser.can_fetch(headers['User-Agent'], uri):
            print(f"Přístup k {uri} není povolen podle robots.txt")
            return

        f = requests.get(uri, headers=headers)
        pol = BeautifulSoup(f.text, 'lxml')

        # Extrahování informací o vozidle (značka, rok vyroby, stav, atd.)
        odkaz = uri  
        auto = pol.find('h1', class_='fs-nejvetsi float-l mb-20').find('b').text
        vyrobeno = pol.findAll('div', class_='radek')[1].findAll('div', class_='sl-6')[1].text
        zaruka = pol.findAll('div', class_='radek')[2].findAll('div', class_='sl-6')[1].text
        stav = ' '.join(pol.findAll('div', class_='radek')[3].findAll('div', class_='sl-6')[1].text.strip().split()).replace('prověřit technikem', '').strip()
        tachometr = ' '.join(pol.findAll('div', class_='radek')[4].findAll('div', class_='sl-6')[1].text.strip().split()).replace('zobrazit historii vozidla', '').strip()
        stk = pol.findAll('div', class_='radek')[5].findAll('div', class_='sl-6')[1].text
        objem = '-'
        vykon = pol.findAll('div', class_='radek')[8].findAll('div', class_='sl-6')[1].text
        moment = pol.findAll('div', class_='radek')[9].findAll('div', class_='sl-6')[1].text
        palivo = pol.findAll('div', class_='radek')[10].findAll('div', class_='sl-6')[1].text
        karoserie = pol.findAll('div', class_='radek')[11].findAll('div', class_='sl-6')[1].text
        pocet_dveri = pol.findAll('div', class_='radek')[12].findAll('div', class_='sl-6')[1].text
        pocet_mist = pol.findAll('div', class_='radek')[13].findAll('div', class_='sl-6')[1].text
        barva = pol.findAll('div', class_='radek')[14].findAll('div', class_='sl-6')[1].text.strip()
        typ = 'Nové'
        cena = pol.find('div', class_='fs-velke').find('b').text
        
        # Vložení dat do data[]
        data.append([odkaz, auto, vyrobeno, zaruka, stav, tachometr, stk, objem, vykon, moment, palivo, karoserie, pocet_dveri, pocet_mist, barva, typ, cena])

        # Zastavení zpracování po dosažení stanoveného limitu
        if len(data) >= limit:
            return

    except Exception as e:
        print(f"Chyba při zpracování stránky {dataLink}: {e}")
        failed_links.append(dataLink)
        
# Procházení odkazů na jednotlivé nabídky paralelně
with ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(process_link, dataLinksLimited)

# Opětovné zpracování odkazů, u kterých došlo k chybě při prvním pokusu
retries = 2 # počet pokusu

for _ in range(retries):
    if not failed_links:
        break

    sleep(3)  # zpoždění

    failed_links_copy = failed_links.copy()
    failed_links = []

    with ThreadPoolExecutor() as executor:
        executor.map(process_link, failed_links_copy)


header = ['odkaz', 'auto', 'vyrobeno', 'záruka', 'stav', 'tachometr', 'STK', 'objem', 'výkon', 'moment', 'palivo', 'karoserie', 'počet_dveří', 'počet_míst', 'barva', 'typ', 'cena']

# Vytvoření DataFrame s použitím načtených dat a zadaných sloupců
df = pd.DataFrame(data, columns=header)

# Uložení DataFrame do souboru CSV
df.to_csv('/Users/adaye/Desktop/tipcars_elektro_nove.csv', sep=';', encoding='utf8')

https://www.tipcars.com/jaguar-i-pace/suv/elektro/jaguar-i-pace-ev400-hse-awd-aut-49548465.htmlhttps://www.tipcars.com/porsche-taycan/kombi/elektro/porsche-taycan-4-cross-turismo-34625232.html
https://www.tipcars.com/bmw-ix/suv/elektro/bmw-ix-xdrive40-12135939.html
https://www.tipcars.com/mercedes-benz-eqe/kupe/elektro/mercedes-benz-eqe-amg-43-4matic-15132896.html
https://www.tipcars.com/mercedes-benz-tridy-v/van/elektro/mercedes-benz-eqv-300-l-fwd-51147077.html
https://www.tipcars.com/jaguar-i-pace/suv/elektro/jaguar-i-pace-ev400-hse-awd-aut-49548860.html
https://www.tipcars.com/bmw-ix/suv/elektro/bmw-ix-xdrive40-26140687.html
https://www.tipcars.com/jaguar-i-pace/suv/elektro/jaguar-i-pace-ev400-hse-awd-aut-49548905.html
https://www.tipcars.com/porsche-taycan/kombi/elektro/porsche-taycan-4s-cross-turismo-34623781.html
https://www.tipcars.com/porsche-taycan/sedan/elektro/porsche-taycan-plus-sport-turismo-34624396.html

https://www.tipcars.com/porsche-taycan/sedan/elektro/porsche-taycan

https://www.tipcars.com/kia-ev6/hatchback/elektro/kia-ev6-0-1-gt-line-5840194.html
https://www.tipcars.com/mercedes-benz-eqa/suv/elektro/mercedes-benz-eqa-300-4matic-15133009.html
https://www.tipcars.com/mercedes-benz-eqa/suv/elektro/mercedes-benz-eqa-250-amg-premium-hud-360-nez-to-37117188.html
https://www.tipcars.com/volvo-c40/suv/elektro/volvo-c40-twin-engine-awd-aut-plus-6548709.html
https://www.tipcars.com/volvo-c40/suv/elektro/volvo-c40-recharge-twin-awd-45782273.html
https://www.tipcars.com/volvo-xc40/suv/elektro/volvo-xc40-recharge-twin-awd-45782251.html
https://www.tipcars.com/kia-ev6/hatchback/elektro/kia-ev6-0-1-gt-line-5840212.html
https://www.tipcars.com/kia-ev6/hatchback/elektro/kia-ev6-0-1-gt-line-5840195.html
https://www.tipcars.com/kia-ev6/hatchback/elektro/kia-ev6-0-1-gt-line-5840213.html
https://www.tipcars.com/skoda-enyaq-iv/suv/elektro/skoda-enyaq-iv-clever-82-kwh-195-kw-4x4-30031432.html
https://www.tipcars.com/kia-ev6/hatchback/elektro/kia-ev6-0-1-gt-line-k-odber